In [0]:
!pip install vecstack

  Created wheel for vecstack: filename=vecstack-0.4.0-cp36-none-any.whl size=19879 sha256=9fc6b8fc3df9b00c3cdafc1f4ee0bf162c094a2967ac5b8dc1e0490e0d0aedc7
  Stored in directory: /root/.cache/pip/wheels/5f/bb/4e/f6488433d53bc0684673d6845e5bf11a25240577c8151c140e
Successfully built vecstack


In [0]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
#from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.svm import SVC
from collections import Counter #for Smote, 

import warnings
warnings.filterwarnings("ignore")


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [0]:
# To upload our datasets from our working directory we need to mount our drive contents to the colab environment. 
# For the code to do so you can search “mount” in code snippets or use the code given below. 
# Our entire drive contents are now mounted on colab at the location “/gdrive”.

from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive


trainfile = r'/gdrive/My Drive/508-A3/newtrain_results_nan.csv'
train_data = pd.read_csv(trainfile)

testfile = r'/gdrive/My Drive/508-A3/newtest_results_nan.csv'
test_data = pd.read_csv(testfile)

print(train_data.shape)
print(train_data.head()) 

print(test_data.shape)
print(test_data.head()) 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive
(260753, 1489)
   Unnamed: 0  QuoteNumber  ...  GeographicField64_TX  QuoteConversion_Flag
0           0            1  ...                     0                     0
1           1            2  ...                     0                     0
2           2            4  ...                     0                     0
3           3            6  ...                     1                     0
4           4            8  ...                     0               

In [0]:

#Copy Train data excluding target
X_train = train_data.iloc[:, :-1].copy()


#Select just Target Column
y_train = train_data.iloc[:, -1]


In [0]:
#CONSTRUCT DEFAULT DECISION TREE AND OBTAIN RESPECTIVE ACCURACY ==================
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
clf_predict=clf.predict(test_data)
#print("accuracy Score (training) for Decision TreE:{0:6f}".format(clf.score(X_test,y_test)))
#print("Confusion Matrix for Decision Tree")
#print(confusion_matrix(y_test,clf_predict))


In [0]:
column1=test_data["QuoteNumber"]

In [0]:
pred=pd.DataFrame(clf_predict,columns=["QuoteConversion_Flag"])
pd.concat([column1,pred],axis=1).to_csv("/gdrive/My Drive/508-A3/DT-Dresults.csv", index = None)

res=pd.read_csv('/gdrive/My Drive/508-A3/DT-Dresults.csv')
res.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,1


In [0]:
#Hyperparameter tuning done for decision tree classifier
parameters={'min_samples_split' : range(10,50,10),'max_depth': range(1,10,2)}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=5)
clf_random.fit(X_train, y_train)
grid_parm=clf_random.best_params_
print(grid_parm)

#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
clf = DecisionTreeClassifier(**grid_parm)
clf.fit(X_train,y_train)


In [0]:
clf_predict = clf.predict(test_data)

pred1=pd.DataFrame(clf_predict,columns=["QuoteConversion_Flag"])
pd.concat([column1,pred1],axis=1).to_csv("/gdrive/My Drive/508-A3/DT-HTresults.csv", index = None)

res=pd.read_csv('/gdrive/My Drive/508-A3/DT-HTresults.csv')
res.head()



,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [0]:
#run cross-validation on best hyperparameters, get auc score
clf_cv_score = cross_val_score(clf, X_train, y_train, cv=5, scoring="roc_auc")
print("=== All AUC Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Decision Tree: ",clf_cv_score.mean())


=== All AUC Scores ===
[0.94369984 0.94484936 0.94827354 0.9460039  0.94640705]


=== Mean AUC Score ===
Mean AUC Score - Decision Tree:  0.9458467374552463


In [0]:
#Random Forest =============================================================
#Default mode
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc_predict=rfc.predict(test_data)

pred2=pd.DataFrame(rfc_predict,columns=["QuoteConversion_Flag"])
pd.concat([column1,pred2],axis=1).to_csv("/gdrive/My Drive/508-A3/RF-Dresults.csv", index = None)

res=pd.read_csv('/gdrive/My Drive/508-A3/RF-Dresults.csv')
res.head()



,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [0]:
#Hyperparameter tuning for random forest
parameters={ 'n_estimators': range(50,100,20),'min_samples_split' : range(10,50,20),'max_depth': range(1,10,2)}
rfc_random = RandomizedSearchCV(rfc,parameters,n_iter=5)
rfc_random.fit(X_train, y_train)
grid_parm_rfc=rfc_random.best_params_
print(grid_parm_rfc)

#contruct random forest using the best parameters
rfc= RandomForestClassifier(**grid_parm_rfc)
rfc.fit(X_train,y_train)
rfc_predict = rfc.predict(test_data)

pred3=pd.DataFrame(rfc_predict,columns=["QuoteConversion_Flag"])
pd.concat([column1,pred3],axis=1).to_csv("/gdrive/My Drive/508-A3/RF-HTresults.csv", index = None)

res=pd.read_csv('/gdrive/My Drive/508-A3/RF-HTresults.csv')
res.head()


#run cross-validation on best parameters, get auc score
rfc_cv_score = cross_val_score(rfc, X_train, y_train, cv=5, scoring="roc_auc")
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ",rfc_cv_score.mean())


{'n_estimators': 90, 'min_samples_split': 10, 'max_depth': 9}
=== All AUC Scores ===
[0.92882141 0.92290742 0.92916421 0.92276619 0.92813533]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.9263589124799367


In [0]:
#Gradient Boosting ============================================================
search_grid={'n_estimators':[20, 30, 50],'learning_rate':[0.01,.1]}
abc =GradientBoostingClassifier()
abc.fit(X_train, y_train)
abc_predict=abc.predict(test_data)

pred4=pd.DataFrame(abc_predict,columns=["QuoteConversion_Flag"])
pd.concat([column1,pred4],axis=1).to_csv("/gdrive/My Drive/508-A3/GB-Dresults.csv", index = None)

res=pd.read_csv('/gdrive/My Drive/508-A3/GB-Dresults.csv')
res.head()


#Randomized Search for hyperparameter tuning
abc_random = RandomizedSearchCV(abc,search_grid,n_iter=5)
abc_random.fit(X_train, y_train)
grid_parm_abc=abc_random.best_params_
print(grid_parm_abc)

{'n_estimators': 50, 'learning_rate': 0.1}


In [0]:
#Construct Gradient Boosting Trees using the best parameters
abc= GradientBoostingClassifier(**grid_parm_abc)
abc.fit(X_train,y_train)
abc_predict = abc.predict(test_data)

pred5=pd.DataFrame(abc_predict,columns=["QuoteConversion_Flag"])
pd.concat([column1,pred5],axis=1).to_csv("/gdrive/My Drive/508-A3/GB-HTresults.csv", index = None)

res=pd.read_csv('/gdrive/My Drive/508-A3/GB-HTresults.csv')
res.head()

#run cross-validation on best parameters, get auc score
abc_cv_score = cross_val_score(abc, X_train, y_train, cv=5, scoring="roc_auc")
print("=== All AUC Scores ===")
print(abc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Boosting: ",abc_cv_score.mean())


=== All AUC Scores ===
[0.95304135 0.95175876 0.95493976 0.95320405 0.95425714]


=== Mean AUC Score ===
Mean AUC Score - Boosting:  0.9534402130806671


In [0]:
#SMOTE==============================================================================
print("___________________________________________________________________\nSMOTE\n")

sm = SMOTE(sampling_strategy='float', ratio=0.5)
X_res, y_res = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res))


___________________________________________________________________
SMOTE

Resampled dataset shape Counter({0: 211859, 1: 105929})


In [0]:
#STACKING MODELS =====================================================================
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier\n")

models = [ GradientBoostingClassifier(n_estimators= 50, learning_rate= 0.1), RandomForestClassifier(n_estimators= 90, min_samples_split= 10, max_depth= 9), DecisionTreeClassifier() ]
      
S_Train, S_Test = stacking(models,                   
                           X_res, y_res, test_data,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=3, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)


___________________________________________________________________________________________
Ensemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [GradientBoostingClassifier]
    fold  0:  [0.93083168]
    fold  1:  [0.93077504]
    fold  2:  [0.93054716]
    ----
    MEAN:     [0.93071796] + [0.00012297]
    FULL:     [0.93071796]

model  1:     [RandomForestClassifier]
    fold  0:  [0.89756443]
    fold  1:  [0.89306146]
    fold  2:  [0.89406012]
    ----
    MEAN:     [0.89489533] + [0.00193087]
    FULL:     [0.89489534]

model  2:     [DecisionTreeClassifier]
    fold  0:  [0.91091287]
    fold  1:  [0.91035590]
    fold  2:  [0.91140208]
    ----
    MEAN:     [0.91089028] + [0.00042740]
    FULL:     [0.91089028]



In [0]:
#STACKING - CONTRUCT A GRADIENT BOOSTING MODEL==============================
model = GradientBoostingClassifier()
    
model = model.fit(S_Train, y_res)
y_pred = model.predict(S_Test)

pred6=pd.DataFrame(y_pred,columns=["QuoteConversion_Flag"])
pd.concat([column1,pred6],axis=1).to_csv("/gdrive/My Drive/508-A3/SMOTE-STACKresults.csv", index = None)

res=pd.read_csv('/gdrive/My Drive/508-A3/SMOTE-STACKresults.csv')
res.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [0]:
#KNN==============================
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=5, weights='uniform', leaf_size=30, p=2, metric='minkowski')
model = model.fit(X_train,y_train)
y_pred = model.predict(test_data)

pred7=pd.DataFrame(y_pred,columns=["QuoteConversion_Flag"])
pd.concat([column1,pred7],axis=1).to_csv("/gdrive/My Drive/508-A3/KNNresults.csv", index = None)

res=pd.read_csv('/gdrive/My Drive/508-A3/KNNresults.csv')
res.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [0]:
from sklearn import svm
print(0)
model =SVC(kernel='linear',max_iter=1)
print(1)
model = model.fit(X_train,y_train)
print(2)
y_pred = model.predict(test_data)
print(3)

pred8=pd.DataFrame(y_pred,columns=["QuoteConversion_Flag"])
pd.concat([column1,pred8],axis=1).to_csv("/gdrive/My Drive/508-A3/SVMresults.csv", index = None)

res=pd.read_csv('/gdrive/My Drive/508-A3/SVMresults.csv')
res.head()

0
1


In [0]:
print(0)
model =MLPClassifier(solver='lbfgs', hidden_layer_sizes=(5, 2))
print(1)
model = model.fit(X_train,y_train)
print(2)
y_pred = model.predict(test_data)
print(3)

pred9=pd.DataFrame(y_pred,columns=["QuoteConversion_Flag"])
pd.concat([column1,pred9],axis=1).to_csv("/gdrive/My Drive/508-A3/MLPresults.csv", index = None)

res=pd.read_csv('/gdrive/My Drive/508-A3/MLPresults.csv')
res.head()

0
1
2
3


,QuoteNumber,QuoteConversion_Flag
0,3,1
1,5,1
2,7,1
3,9,1
4,10,1


In [0]:
print(0)
model =MLPClassifier(solver='lbfgs', hidden_layer_sizes=(100, 10))
print(1)
model = model.fit(X_res,y_res)
print(2)
y_pred = model.predict(test_data)
print(3)

pred10=pd.DataFrame(y_pred,columns=["QuoteConversion_Flag"])
pd.concat([column1,pred10],axis=1).to_csv("/gdrive/My Drive/508-A3/MLP-SMOTEresults.csv", index = None)

res=pd.read_csv('/gdrive/My Drive/508-A3/MLP-SMOTEresults.csv')
res.head()

0
1
2
3


,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [0]:
print(0)
model =MLPClassifier()
print(1)
model = model.fit(X_res,y_res)
print(2)
y_pred = model.predict(test_data)
print(3)

pred11=pd.DataFrame(y_pred,columns=["QuoteConversion_Flag"])
pd.concat([column1,pred11],axis=1).to_csv("/gdrive/My Drive/508-A3/MLP-SMOTE-Dresults.csv", index = None)

res=pd.read_csv('/gdrive/My Drive/508-A3/MLP-SMOTE-Dresults.csv')
res.head()

0
1
2
3


,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [0]:
#SMOTE2==============================================================================
print("___________________________________________________________________\nSMOTE\n")

sm = SMOTE(sampling_strategy='float', ratio=0.3)
X_res, y_res = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res))

___________________________________________________________________
SMOTE

Resampled dataset shape Counter({0: 211859, 1: 63557})


In [0]:
#STACKING MODELS 2 =====================================================================
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier\n")

models = [ GradientBoostingClassifier(n_estimators= 50, learning_rate= 0.1), RandomForestClassifier(n_estimators= 90, min_samples_split= 10, max_depth= 9), DecisionTreeClassifier() ]
      
S_Train, S_Test = stacking(models,                   
                           X_res, y_res, test_data,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=3, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)

___________________________________________________________________________________________
Ensemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [GradientBoostingClassifier]
    fold  0:  [0.92334924]
    fold  1:  [0.92233623]
    fold  2:  [0.92322775]
    ----
    MEAN:     [0.92297107] + [0.00045163]
    FULL:     [0.92297107]

model  1:     [RandomForestClassifier]
    fold  0:  [0.87026992]
    fold  1:  [0.87313465]
    fold  2:  [0.87504902]
    ----
    MEAN:     [0.87281786] + [0.00196388]
    FULL:     [0.87281785]

model  2:     [DecisionTreeClassifier]
    fold  0:  [0.89690216]
    fold  1:  [0.89666253]
    fold  2:  [0.89833776]
    ----
    MEAN:     [0.89730082] + [0.00073973]
    FULL:     [0.89730081]



In [0]:
#STACKING - CONTRUCT A RANDOM FOREST MODEL==============================
#Hyperparameter tuning for random forest
rfc = RandomForestClassifier()
parameters={ 'n_estimators': range(50,100,10),'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}
rfc_random = RandomizedSearchCV(rfc,parameters,n_iter=10)
rfc_random.fit(S_Train, y_res)
grid_parm_rfc=rfc_random.best_params_
print(grid_parm_rfc)

#contruct random forest using the best parameters
rfc= RandomForestClassifier(**grid_parm_rfc)
rfc.fit(S_Train, y_res)
rfc_predict = rfc.predict(S_Test)


pred12=pd.DataFrame(rfc_predict,columns=["QuoteConversion_Flag"])
pd.concat([column1,pred12],axis=1).to_csv("/gdrive/My Drive/508-A3/SMOTE0.3-STACK-RFresults.csv", index = None)

res=pd.read_csv('/gdrive/My Drive/508-A3/SMOTE0.3-STACK-RFresults.csv')
res.head()


#run cross-validation on best parameters, get auc score
rfc_cv_score = cross_val_score(rfc, S_Train, y_res, cv=5, scoring="roc_auc")
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ",rfc_cv_score.mean())


{'n_estimators': 80, 'min_samples_split': 60, 'max_depth': 11}
=== All AUC Scores ===
[0.85964    0.8594937  0.86090165 0.88079    0.9974819 ]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.8916614511491152
